In [ ]:
import pickle
import json
from flask import Flask, request
import numpy as np
service = Flask(__name__)

##################

def unpickle():
    global trained
    global classes
    repo='/home/jovyan/work/dataismus/di-interview-product-classifier/training/'
    with open(repo+'trained.pkl', 'rb') as f: trained = pickle.load(f)
    classes=trained.classes_

##################

@service.route('/classify', methods=['POST'])
def get_prediction():
    data = request.get_json()
    probas=trained.predict_proba([data["title"]])[0]
    top5=np.argsort(probas)[-5:]
        
    #requested JSON elements:
    title=data["title"]
    top_5_results=[{"product_type": x, "score": "{:.4f}".format(y)} for (x,y) in zip(reversed(classes[top5]),reversed(probas[top5]))]
    product_type=classes[top5][-1]
    return json.dumps({"title": title, "top_5_results": top_5_results, "product_type": product_type}, indent=3)

##################

if __name__ == '__main__':
    unpickle()
    service.run(host='0.0.0.0', port=8055)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8055/ (Press CTRL+C to quit)
127.0.0.1 - - [26/May/2020 18:40:47] "POST /classify HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2020 18:41:15] "POST /classify HTTP/1.1" 200 -
172.17.0.1 - - [26/May/2020 18:41:27] "GET / HTTP/1.1" 404 -
172.17.0.1 - - [26/May/2020 18:41:27] "GET /favicon.ico HTTP/1.1" 404 -
172.17.0.1 - - [26/May/2020 18:41:32] "GET /classify HTTP/1.1" 405 -
172.17.0.1 - - [26/May/2020 18:42:00] "GET /classify HTTP/1.1" 405 -
172.17.0.1 - - [26/May/2020 18:42:18] "GET /classify HTTP/1.1" 405 -
172.17.0.1 - - [26/May/2020 18:42:23] "GET /classify/batman HTTP/1.1" 404 -
172.17.0.1 - - [26/May/2020 18:42:29] "GET /classify/%22batman%22 HTTP/1.1" 404 -
172.17.0.1 - - [26/May/2020 18:42:33] "GET /classify/ HTTP/1.1" 404 -
172.17.0.1 - - [26/May/2020 18:47:53] "POST /classify HTTP/1.1" 200 -
172.17.0.1 - - [26/May/2020 18:47:55] "POST /classify HTTP/1.1" 200 -
172.17.0.1 - - [26/May/2020 18:49:20] "POST /classify HTTP/1.1" 200 -
172.17.0.1 - - [26/May/202

In [ ]:
# curl -XPOST 'http://localhost:5050/classify' -H 'Content-Type: application/json' -d '{ "title": "specialized batman" }'